In [1]:
import pandas as pd
import numpy as np
import tqdm
import os
import glob
import cv2

## Dataloader

### View Matching

In [2]:
import tensorflow as tf

In [13]:
train = pd.read_csv("/content/drive/My Drive/AG-CNN/dataset/test_set_both.csv")

In [ ]:
len(train[train['ViewPosition'] == 'LL'])

5403

In [14]:
def generate_path(row):
  folder = str(int(row['subject_id']))
  study_id = str(int(row['study_id']))
  dicom = row['dicom_id']
  return '/content/drive/My Drive/mimic-cxr/images512/p{}/p{}/s{}/{}.png'.format(
      folder[:2],
      folder,
      study_id,
      dicom
  )

In [15]:
train['path'] = train.apply(lambda x : generate_path(x), axis=1)

In [16]:
train_g = train.groupby('study_id')

In [17]:
import random 

def aggregate_images(df):
  if len(df) > 1:
    pa = df[(df['ViewPosition'] == 'PA')]['path']
    ap = df[(df['ViewPosition'] == 'AP')]['path']
    ll =  df[(df['ViewPosition'] == 'LL')]['path']
    la =  df[(df['ViewPosition'] == 'LATERAL')]['path']
    views = [ap, pa, ll, la]
    if (len(pa) == 0 and len(ap) == 0) or (len(ll) == 0 and len(la) == 0):
      return None
    
    result_frontal = []
    result_side = []
    views = []

    for path in ap.values:
      for path_l in ll.values:
        result_frontal.append(path)
        result_side.append(path_l)
        views.append('ap_ll')
        
      for path_l in la.values:
        result_frontal.append(path)
        result_side.append(path_l)
        views.append('ap_la')
    
    for path in pa.values:
      for path_l in ll.values:
        result_frontal.append(path)
        result_side.append(path_l)
        views.append('pa_ll')
        
      for path_l in la.values:
        result_frontal.append(path)
        result_side.append(path_l)
        views.append('pa_la')

    selected = random.randint(0, len(views) - 1)
    select = np.zeros(len(views), dtype=np.int8).tolist()
    select[selected] = 1

    pre_result = pd.DataFrame({
          'study_id' : df.iloc[0, 3],
          'frontal': result_frontal,
          'side': result_side,
          'view': views,
          'select': select
      }, columns = ['study_id', 'frontal','side', 'view', 'select'])
    return pre_result
  return None

In [18]:
train_g = train_g.apply(aggregate_images)

In [19]:
train_g.reset_index(drop=True, inplace=True)

In [20]:
train_l = train.groupby('study_id')

In [21]:
train_l = train_l.first()

In [22]:
train_merge = pd.merge(train_l, train_g,left_on='study_id', right_on='study_id', how='right')

In [ ]:
train['ViewPosition'].value_counts()

AP          75080
PA          66968
LATERAL     58130
LL          24302
LAO             3
RAO             3
PA LLD          2
AP AXIAL        2
AP LLD          2
AP RLD          1
SWIMMERS        1
Name: ViewPosition, dtype: int64

In [ ]:
train_merge['view'].value_counts()

pa_la    48147
pa_ll    22473
ap_la    17035
ap_ll     2130
Name: view, dtype: int64

In [24]:
train_merge.to_csv("/content/drive/My Drive/AG-CNN/dataset/test_set_merge.csv", index=False, header=True)

In [23]:
train_merge.head()

,study_id,Unnamed: 0,dicom_id,subject_id,ViewPosition,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices,path,frontal,side,view,select
0,50000028,376819,1ff9650f-4cb5af18-f6caef33-e53686b7-983cca76,19995127,LATERAL,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/My Drive/mimic-cxr/images512/p1...,/content/drive/My Drive/mimic-cxr/images512/p1...,/content/drive/My Drive/mimic-cxr/images512/p1...,pa_la,1
1,50000230,57297,7e962a95-d661c0db-4769286c-e150a106-fb9586c6,11550925,PA,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/My Drive/mimic-cxr/images512/p1...,/content/drive/My Drive/mimic-cxr/images512/p1...,/content/drive/My Drive/mimic-cxr/images512/p1...,pa_la,1
2,50000766,37737,5fbd6d46-6ebe9e7e-ece0d766-43ebe17f-c2605f24,11027112,PA,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,/content/drive/My Drive/mimic-cxr/images512/p1...,/content/drive/My Drive/mimic-cxr/images512/p1...,/content/drive/My Drive/mimic-cxr/images512/p1...,pa_ll,1
3,50000936,75474,1b4edfd9-223f91d2-eb1ec8da-f98e2968-15ead25f,12008045,LATERAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,/content/drive/My Drive/mimic-cxr/images512/p1...,/content/drive/My Drive/mimic-cxr/images512/p1...,/content/drive/My Drive/mimic-cxr/images512/p1...,pa_la,1
4,50001481,221570,02d065af-d0785395-162a5490-e0e1143e-d24656af,15868528,LATERAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,/content/drive/My Drive/mimic-cxr/images512/p1...,/content/drive/My Drive/mimic-cxr/images512/p1...,/content/drive/My Drive/mimic-cxr/images512/p1...,pa_la,1


In [ ]:
train_labels = train_merge.iloc[:, 5:19]

### Image Pre-processing

In [ ]:
# train_la = train[(train['ViewPosition'] == 'LL')]

In [ ]:
# train_labels = train_la.iloc[:,5:19]

In [ ]:
# train_labels.fillna(0, inplace=True)

# train_labels.replace(-1, 0, inplace=True)

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((train_la['path'],np.array(train_labels), train_la['study_id']))

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((train_merge['study_id'],train_merge['frontal'], train_merge['side'], train_merge['view'],train_merge['select'], np.array(train_labels)))

In [ ]:
def decode_img(img):
  img = tf.image.decode_png(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)
  return tf.image.resize(img, (224,224))

def process_path(path, label, sid):
  img = tf.io.read_file(path)
  img = decode_img(img)
  return img, label, sid

def process_path_mul(sid, front, side, view, select, label):
  img_front = tf.io.read_file(front)
  img_front = decode_img(img_front)

  img_side = tf.io.read_file(side)
  img_side = decode_img(img_side)

  return img_front, img_side, label, view, select, sid

In [ ]:
train_im = train_ds.map(process_path, num_parallel_calls=tf.data.experimental.AUTOTUNE)\
    .apply(tf.data.experimental.ignore_errors()).cache('/content/train_cache')

In [ ]:
train_im = train_ds.map(process_path_mul, num_parallel_calls=tf.data.experimental.AUTOTUNE)\
    .apply(tf.data.experimental.ignore_errors()).cache('/content/train_cache')

### Create TFRecord

In [ ]:
len(train_merge)

19122

In [ ]:
BS = 64
# TR_SHARDS = 10
TR_SHARDS = 10
GCS_OUTPUT = '/content/drive/My Drive/AG-CNN/tfrecord_train_mul_nr/' 

In [ ]:
import math

tr_size = len(train_merge)
tr_shard_size = math.ceil(1.0 * tr_size / TR_SHARDS)

In [ ]:
tr_shard_size

2245

In [ ]:
def recompress_image(image, label, sid):
  image = image * 255
  image = tf.cast(image, tf.uint8)
  image = tf.image.encode_jpeg(image, optimize_size=True, chroma_downsampling=False)
  return image, label, sid

In [ ]:
def recompress_image_mul(image_front,image_side, label, view, select, sid):
  image_front = image_front * 255
  image_front = tf.cast(image_front, tf.uint8)
  image_front = tf.image.encode_jpeg(image_front, optimize_size=True, chroma_downsampling=False)

  image_side = image_side * 255
  image_side = tf.cast(image_side, tf.uint8)
  image_side = tf.image.encode_jpeg(image_side, optimize_size=True, chroma_downsampling=False)
  return image_front, image_side, label, view, select, sid

In [ ]:
train_imc = train_im.map(recompress_image_mul, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_imc = train_imc.batch(tr_shard_size).prefetch(1)

In [ ]:
# val_imc = val_im.map(recompress_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
# val_imc = val_imc.batch(val_shard_size).prefetch(1)

In [ ]:
# Three types of data can be stored in TFRecords: bytestrings, integers and floats
# They are always stored as lists, a single data element will be a list of size 1

def _bytestring_feature(list_of_bytestrings):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=list_of_bytestrings))

def _int_feature(list_of_ints): # int64
  return tf.train.Feature(int64_list=tf.train.Int64List(value=list_of_ints))

def _float_feature(list_of_floats): # float32
  return tf.train.Feature(float_list=tf.train.FloatList(value=list_of_floats))
  

def to_tfrecord(tfrec_filewriter, img_front,img_side, label, view, select, sid):  
  
  feature = {
      "image_front": _bytestring_feature([img_front]), 
      "image_side": _bytestring_feature([img_side]), 
      "label":         _float_feature(label.reshape(-1)), 
      "view": _bytestring_feature([view]), 
      "select": _int_feature([select]),
      "study_id": _int_feature([sid])
    }
  return tf.train.Example(features=tf.train.Features(feature=feature))


In [ ]:
print("Writing TFRecords")
for shard, (image_front, image_la, label,view, select, sid) in tqdm.tqdm(enumerate(train_imc, start=19)):
  # batch size used as shard size here
  shard_size = image_front.numpy().shape[0]
  # good practice to have the number of records in the filename
  filename = GCS_OUTPUT + "{:02d}-{}.tfrec".format(shard, shard_size)
  if os.path.exists(filename): 
    continue
  
  with tf.io.TFRecordWriter(filename) as out_file:
    for i in tqdm.tqdm(range(shard_size)):
      example = to_tfrecord(out_file,
                            image_front.numpy()[i],
                            image_la.numpy()[i],
                            label.numpy()[i],
                            view.numpy()[i],
                            select.numpy()[i],
                            sid.numpy()[i])
      out_file.write(example.SerializeToString())
    print("Wrote file {} containing {} records".format(filename, shard_size))

0it [00:00, ?it/s]

Writing TFRecords



100%|██████████| 2245/2245 [11:42<00:00,  3.19it/s]
1it [2:20:43, 8443.35s/it]

Wrote file /content/drive/My Drive/AG-CNN/tfrecord_train_mul_nr/19-2245.tfrec containing 2245 records



100%|██████████| 2245/2245 [11:17<00:00,  3.31it/s]
2it [3:20:51, 6992.75s/it]

Wrote file /content/drive/My Drive/AG-CNN/tfrecord_train_mul_nr/20-2245.tfrec containing 2245 records



100%|██████████| 2245/2245 [11:35<00:00,  3.23it/s]
3it [4:19:47, 5955.68s/it]

Wrote file /content/drive/My Drive/AG-CNN/tfrecord_train_mul_nr/21-2245.tfrec containing 2245 records



100%|██████████| 2245/2245 [12:20<00:00,  3.03it/s]
4it [5:21:47, 5285.07s/it]

Wrote file /content/drive/My Drive/AG-CNN/tfrecord_train_mul_nr/22-2245.tfrec containing 2245 records


In [ ]:
def read_tfrecord(example):
    features = {
        "image": tf.io.FixedLenFeature([], tf.string),  # tf.string = bytestring (not text string)
        "label": tf.io.VarLenFeature(tf.float32)        # a certain number of floats0
        "size":  tf.io.FixedLenFeature([2], tf.int64),  # two integers
    }
    # decode the TFRecord
    example = tf.io.parse_single_example(example, features)
    
    # FixedLenFeature fields are now ready to use: exmple['size']
    # VarLenFeature fields require additional sparse_to_dense decoding
    
    image = tf.image.decode_jpeg(example['image'], channels=3)
    label  = tf.reshape(tf.sparse.to_dense(example['label']), [-1, 14])
    height = example['size'][0]
    width  = example['size'][1]
    image = tf.reshape(image, [height, width, 3])
    return image, label

In [ ]:
# read from TFRecords. For optimal performance, read from multiple
# TFRecord files at once and set the option experimental_deterministic = False
# to allow order-altering optimizations.

option_no_order = tf.data.Options()
option_no_order.experimental_deterministic = False

filenames = tf.io.gfile.glob(GCS_OUTPUT + "*.tfrec")
train_dsr = tf.data.TFRecordDataset(filenames, num_parallel_reads=tf.data.experimental.AUTOTUNE)
train_dsr = train_dsr.with_options(option_no_order)
train_dsr = train_dsr.map(read_tfrecord, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dsr = train_dsr.shuffle(300)